In [2]:

from utils_aa import * 

study_real = optuna.create_study(
    study_name='study_real',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True, 
)


study_real_sampling = optuna.create_study(
    study_name='study_real_sampling',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True,
)



make_pareto_plot(study_real_sampling)
make_ranking_plots(study_real_sampling, get_median_dict(study_real), data ="_best_artifact/optuna/data.csv", CUTOFF_AUC =.9, CUTOFF_PHE=.0)

[I 2023-05-03 21:01:11,157] Using an existing study with name 'study_real' instead of creating a new one.
[I 2023-05-03 21:01:11,175] Using an existing study with name 'study_real_sampling' instead of creating a new one.


{'best_iteration': '39', 'best_ntree_limit': '40', 'best_score': '0.08114367102583249'}
Phe_value :  0.15181865 	 Phe_ranking :  15.0


{'best_iteration': '2', 'best_ntree_limit': '3', 'best_score': '0.42957672476768494'}
Phe_value :  0.21201801 	 Phe_ranking :  8.0


{'best_iteration': '20', 'best_ntree_limit': '21', 'best_score': '0.21961123868823051'}
Phe_value :  0.25939038 	 Phe_ranking :  7.0


{'best_iteration': '22', 'best_ntree_limit': '23', 'best_score': '0.3154486215983828'}
Phe_value :  0.30960056 	 Phe_ranking :  5.0


{'best_iteration': '14', 'best_ntree_limit': '15', 'best_score': '0.35881624184548855'}
Phe_value :  0.22882034 	 Phe_ranking :  9.0


{'best_iteration': '11', 'best_ntree_limit': '12', 'best_score': '0.4599017488459746'}
Phe_value :  0.36585155 	 Phe_ranking :  6.0


{'best_iteration': '25', 'best_ntree_limit': '26', 'best_score': '0.1610239134170115'}
Phe_value :  0.25733283 	 Phe_ranking :  10.0


{'best_iteration': '4', 'best_ntree_limit': '5', 'best_sco

# ----------------------------------------------------

In [ ]:

#study_name: str = "alejandro_resampled_data_BorderlineSMOTE"

study_BorderlineSMOTE = optuna.create_study(
    study_name='study_BorderlineSMOTE',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True,
)

study_SVMSMOTE = optuna.create_study(
    study_name='study_SVMSMOTE',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True,
)


study_ADASYN = optuna.create_study(
    study_name='study_ADASYN',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True,
)

study_real = optuna.create_study(
    study_name='study_real',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True, 
)

# ########################
# study_real.optimize(lambda trial: objective(trial, "_best_artifact/optuna/data.csv",                                                        finetunning = True), n_trials=5000, n_jobs=-1)
# study_BorderlineSMOTE.optimize(lambda trial: objective(trial, "_best_artifact/optuna/resampled_data_BorderlineSMOTE.csv",                   finetunning = True), n_trials=5000, n_jobs=-1)
# study_SVMSMOTE.optimize(lambda trial: objective(trial, "_best_artifact/optuna/resampled_data_SVMSMOTE.csv",                                 finetunning = True), n_trials=5000, n_jobs=-1)
# study_ADASYN.optimize(lambda trial: objective(trial, "_best_artifact/optuna/resampled_data_ADASYN.csv",                                     finetunning = True), n_trials=5000, n_jobs=-1)
# ###################
#make_pareto_plot(study_real)


In [2]:

######## SAMPLING ########

study_BorderlineSMOTE_sampling = optuna.create_study(
    study_name='study_BorderlineSMOTE_sampling',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True,
)

study_SVMSMOTE_sampling = optuna.create_study(
    study_name='study_SVMSMOTE_sampling',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True,
)



study_ADASYN_sampling = optuna.create_study(
    study_name='study_ADASYN_sampling',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True,
)

study_real_sampling = optuna.create_study(
    study_name='study_real_sampling',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True,
)


[I 2023-05-03 20:49:37,279] Using an existing study with name 'study_BorderlineSMOTE_sampling' instead of creating a new one.
[I 2023-05-03 20:49:37,296] Using an existing study with name 'study_SVMSMOTE_sampling' instead of creating a new one.
[I 2023-05-03 20:49:37,314] Using an existing study with name 'study_ADASYN_sampling' instead of creating a new one.
[I 2023-05-03 20:49:37,330] Using an existing study with name 'study_real_sampling' instead of creating a new one.


In [3]:



# study_real_sampling.optimize(lambda trial: objective(trial, "_best_artifact/optuna/data.csv",  tuned_params = get_median_dict(study_real),                    finetunning = False), n_trials=1000, n_jobs=-1)
# study_ADASYN_sampling.optimize(lambda trial: objective(trial, "_best_artifact/optuna/resampled_data_ADASYN.csv",  tuned_params = get_median_dict(study_ADASYN),                    finetunning = False), n_trials=1000, n_jobs=-1)
# study_BorderlineSMOTE_sampling.optimize(lambda trial: objective(trial, "_best_artifact/optuna/resampled_data_BorderlineSMOTE.csv",  tuned_params = get_median_dict(study_BorderlineSMOTE),  finetunning = False), n_trials=1000, n_jobs=-1)
# study_SVMSMOTE_sampling.optimize(lambda trial: objective(trial, "_best_artifact/optuna/resampled_data_SVMSMOTE.csv",  tuned_params = get_median_dict(study_SVMSMOTE),                finetunning = False), n_trials=1000, n_jobs=-1)




In [18]:
make_pareto_plot(study_real_sampling)
make_ranking_plots(study_real_sampling, get_median_dict(study_real), data ="_best_artifact/optuna/data.csv", CUTOFF_AUC =.9, CUTOFF_PHE=.0)

{'best_iteration': '39', 'best_ntree_limit': '40', 'best_score': '0.08114367102583249'}
Phe_value :  0.15181865 	 Phe_ranking :  15.0


{'best_iteration': '2', 'best_ntree_limit': '3', 'best_score': '0.42957672476768494'}
Phe_value :  0.21201801 	 Phe_ranking :  8.0


{'best_iteration': '20', 'best_ntree_limit': '21', 'best_score': '0.21961123868823051'}
Phe_value :  0.25939038 	 Phe_ranking :  7.0


{'best_iteration': '22', 'best_ntree_limit': '23', 'best_score': '0.3154486215983828'}
Phe_value :  0.30960056 	 Phe_ranking :  5.0


{'best_iteration': '14', 'best_ntree_limit': '15', 'best_score': '0.35881624184548855'}
Phe_value :  0.22882034 	 Phe_ranking :  9.0


{'best_iteration': '11', 'best_ntree_limit': '12', 'best_score': '0.4599017488459746'}
Phe_value :  0.36585155 	 Phe_ranking :  6.0


{'best_iteration': '25', 'best_ntree_limit': '26', 'best_score': '0.1610239134170115'}
Phe_value :  0.25733283 	 Phe_ranking :  10.0


{'best_iteration': '4', 'best_ntree_limit': '5', 'best_sco

In [13]:
make_pareto_plot(study_ADASYN_sampling)
make_ranking_plots(study_ADASYN_sampling, get_median_dict(study_ADASYN), "_best_artifact/optuna/resampled_data_ADASYN.csv", CUTOFF_AUC =.9, CUTOFF_PHE=.3)

{'best_iteration': '35', 'best_ntree_limit': '36', 'best_score': '0.16127585377544165'}
Phe_value :  0.6797707 	 Phe_ranking :  1.0


{'best_iteration': '33', 'best_ntree_limit': '34', 'best_score': '0.4396458752453327'}
Phe_value :  0.31102696 	 Phe_ranking :  7.0


{'best_iteration': '16', 'best_ntree_limit': '17', 'best_score': '0.3229215431958437'}
Phe_value :  0.3209652 	 Phe_ranking :  7.0


{'best_iteration': '41', 'best_ntree_limit': '42', 'best_score': '0.10036763716489076'}
Phe_value :  0.33645225 	 Phe_ranking :  6.0


{'best_iteration': '14', 'best_ntree_limit': '15', 'best_score': '0.2303225699812174'}
Phe_value :  0.3155051 	 Phe_ranking :  5.0


{'best_iteration': '48', 'best_ntree_limit': '49', 'best_score': '0.11369716059416532'}
Phe_value :  0.4261694 	 Phe_ranking :  4.0


{'best_iteration': '62', 'best_ntree_limit': '63', 'best_score': '0.08558865338563919'}
Phe_value :  0.30214596 	 Phe_ranking :  6.0


{'best_iteration': '11', 'best_ntree_limit': '12', 'best_score

In [14]:
make_pareto_plot(study_BorderlineSMOTE_sampling)
make_ranking_plots(study_BorderlineSMOTE_sampling, get_median_dict(study_BorderlineSMOTE),data="_best_artifact/optuna/resampled_data_BorderlineSMOTE.csv" ,CUTOFF_AUC =.9, CUTOFF_PHE=.3)



{'best_iteration': '32', 'best_ntree_limit': '33', 'best_score': '0.22329338089647618'}
Phe_value :  0.35435537 	 Phe_ranking :  3.0


{'best_iteration': '44', 'best_ntree_limit': '45', 'best_score': '0.2596135400235653'}
Phe_value :  0.3095363 	 Phe_ranking :  5.0


{'best_iteration': '49', 'best_ntree_limit': '50', 'best_score': '0.1968060571023009'}
Phe_value :  0.66210276 	 Phe_ranking :  2.0


{'best_iteration': '35', 'best_ntree_limit': '36', 'best_score': '0.11651333048939705'}
Phe_value :  0.3219652 	 Phe_ranking :  7.0


{'best_iteration': '6', 'best_ntree_limit': '7', 'best_score': '0.5863465917381373'}
Phe_value :  0.32645813 	 Phe_ranking :  5.0


{'best_iteration': '14', 'best_ntree_limit': '15', 'best_score': '0.5389901108362458'}
Phe_value :  0.6428246 	 Phe_ranking :  2.0


{'best_iteration': '53', 'best_ntree_limit': '54', 'best_score': '0.18367204705083912'}
Phe_value :  0.3097634 	 Phe_ranking :  7.0


{'best_iteration': '11', 'best_ntree_limit': '12', 'best_score': 

In [17]:

make_pareto_plot(study_SVMSMOTE_sampling)
make_ranking_plots(study_SVMSMOTE_sampling, get_median_dict(study_SVMSMOTE), data="_best_artifact/optuna/resampled_data_SVMSMOTE.csv",CUTOFF_AUC =.9, CUTOFF_PHE=.1)

/DeepenData/Repos/Phenylketonuria_IR_and_Machine_Learning/utils_aa.py:339: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/DeepenData/Repos/Phenylketonuria_IR_and_Machine_Learning/utils_aa.py:339: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/DeepenData/Repos/Phenylketonuria_IR_and_Machine_Learning/utils_aa.py:339: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.

{'best_iteration': '16', 'best_ntree_limit': '17', 'best_score': '0.30310510388678974'}
Phe_value :  0.16971344 	 Phe_ranking :  8.0


{'best_iteration': '51', 'best_ntree_limit': '52', 'best_score': '0.35117582562896943'}
Phe_value :  0.22950834 	 Phe_ranking :  7.0


{'best_iteration': '31', 'best_ntree_limit': '32', 'best_score': '0.409355228766799'}
Phe_value :  0.14808255 	 Phe_ranking :  13.0


{'best_iteration': '13', 'best_ntree_limit': '14', 'best_score': '0.4002406911717521'}
Phe_value :  0.1878913 	 Phe_ranking :  8.0


{'best_iteration': '15', 'best_ntree_limit': '16', 'best_score': '0.27847034608324367'}
Phe_value :  0.23124078 	 Phe_ranking :  7.0


{'best_iteration': '54', 'best_ntree_limit': '55', 'best_score': '0.1330690568106042'}
Phe_value :  0.18097325 	 Phe_ranking :  10.0


{'best_iteration': '28', 'best_ntree_limit': '29', 'best_score': '0.2181183377073871'}
Phe_value :  0.3499286 	 Phe_ranking :  5.0


{'best_iteration': '64', 'best_ntree_limit': '65', 'best_sco

/DeepenData/Repos/Phenylketonuria_IR_and_Machine_Learning/utils_aa.py:371: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/DeepenData/Repos/Phenylketonuria_IR_and_Machine_Learning/utils_aa.py:371: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



{'best_iteration': '9', 'best_ntree_limit': '10', 'best_score': '0.3888152572843764'}
Phe_value :  0.39988667 	 Phe_ranking :  4.0


{'best_iteration': '5', 'best_ntree_limit': '6', 'best_score': '0.41014159553580815'}
Phe_value :  0.19120961 	 Phe_ranking :  5.0


{'best_iteration': '38', 'best_ntree_limit': '39', 'best_score': '0.421683881431818'}
Phe_value :  0.30637327 	 Phe_ranking :  4.0


{'best_iteration': '16', 'best_ntree_limit': '17', 'best_score': '0.43532714454664123'}
Phe_value :  0.7859423 	 Phe_ranking :  1.0




/DeepenData/Repos/Phenylketonuria_IR_and_Machine_Learning/utils_aa.py:371: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/DeepenData/Repos/Phenylketonuria_IR_and_Machine_Learning/utils_aa.py:371: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



...

/DeepenData/Repos/Phenylketonuria_IR_and_Machine_Learning/utils_aa.py:371: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



{'best_iteration': '46', 'best_ntree_limit': '47', 'best_score': '0.23105306488772234'}
Phe_value :  0.17265882 	 Phe_ranking :  11.0


{'best_iteration': '18', 'best_ntree_limit': '19', 'best_score': '0.2603790366815196'}
Phe_value :  0.21417059 	 Phe_ranking :  5.0




/DeepenData/Repos/Phenylketonuria_IR_and_Machine_Learning/utils_aa.py:371: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/DeepenData/Repos/Phenylketonuria_IR_and_Machine_Learning/utils_aa.py:371: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



{'best_iteration': '24', 'best_ntree_limit': '25', 'best_score': '0.06798687287502819'}
Phe_value :  0.1972442 	 Phe_ranking :  7.0




/DeepenData/Repos/Phenylketonuria_IR_and_Machine_Learning/utils_aa.py:371: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/DeepenData/.miniconda/envs/playground/lib/python3.10/site-packages/plotly/express/_core.py:1223: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/DeepenData/.miniconda/envs/playground/lib/python3.10/site-packages/plotly/express/_core.py:1223: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all co